In [ ]:
import os
import pprint
import glob
from tqdm import tqdm
import ffmpeg
import subprocess
import exiftool
import subprocess
import shutil
from datetime import datetime
from pathlib import Path

In [ ]:
INPUT_CAMERA = Path(os.getenv("CAMERA_INPUT"))
OUTPUT_VIDEO_CAMERA = Path(os.getenv("VIDEO_COMPRESSED"))
if not OUTPUT_VIDEO_CAMERA.exists():
    os.makedirs(OUTPUT_VIDEO_CAMERA)

input_camera_video_paths = [
    Path(path) for path in glob.glob(str(INPUT_CAMERA / "*.mp4"))
]
input_camera_video_paths = sorted(
    input_camera_video_paths, key=os.path.getsize, reverse=True
)
input_camera_video_paths[0:5]

In [ ]:
for input_path in (bar := tqdm(input_camera_video_paths)):
    output_path = OUTPUT_VIDEO_CAMERA / input_path.name
    if output_path.exists():
        continue

    msg = f"{input_path.name} from {os.path.getsize(input_path) // (1024)**2}Mb to..."
    bar.set_description(msg)

    with exiftool.ExifToolHelper() as input_exiftool:
        input_metadata = input_exiftool.get_metadata(input_path)[0]

    keep_metadata = {
        key: value
        for key, value in input_metadata.items()
        if (
            key
            in [
                "QuickTime:GPSCoordinates",
            ]
        )
        or ("Date" in key)
        or ("Create" in key)
    }
    
    create_date_str = (
        str(keep_metadata["QuickTime:CreateDate"])
        if str(keep_metadata["QuickTime:CreateDate"]).startswith("20")
        else str(keep_metadata["File:FileModifyDate"])
    )
    creatredate = datetime.strptime(create_date_str.split("+")[0], "%Y:%m:%d %H:%M:%S")
    date_str_touch = creatredate.strftime("%m/%d/%Y %H:%M:%S")
    if not output_path.exists():
        (
            ffmpeg.input(str(input_path))
            .output(
                str(output_path),
                **{
                    "c:v": "libx265",
                    "c:a": "aac",
                    "b:a": "128k",
                    "tag:v": "hvc1",
                },
            )
            .overwrite_output()
            .run(capture_stdout=True, capture_stderr=True)
        )
        with exiftool.ExifToolHelper(
            common_args=["-overwrite_original"]
        ) as output_exiftool:
            output_exiftool.set_tags(
                output_path,
                keep_metadata,
            )
        subprocess.check_call(["SetFile", "-d", date_str_touch, str(output_path)])

    end_msg = f"{msg} -> {os.path.getsize(output_path) // (1024)**2}Mb"
    bar.set_description(end_msg)
    print(end_msg)
    if os.path.getsize(output_path) > os.path.getsize(input_path):
        print("Output bigger than input, removing")
        os.remove(output_path)
        # shutil.copy(input_path, output_path)